In [1]:
!echo "PYTHONPATH: $PYTHONPATH || GDAL:  $GDAL_DATA"
!which python

import sys
paths=sys.path
#paths.append('/g/data1/u46/fxz547/Githubz/agdc-v2')
paths.insert(0,'/g/data1/u46/fxz547/Githubz/agdc-v2')  #prepend a path
for p in  sys.path:
    print p

PYTHONPATH: /projects/v10/NEMO/hpc-lpgs/python: || GDAL:  /g/data/u46/fxz547/anaconda2/share/gdal/
/g/data/u46/fxz547/anaconda2/bin/python
/g/data1/u46/fxz547/Githubz/agdc-v2

/g/data/u46/fxz547/anaconda2/lib/python2.7/site-packages/setuptools-20.2.2-py2.7.egg
/projects/v10/NEMO/hpc-lpgs/python
/g/data1/u46/fxz547/Githubz/wofs/Zlocaltest
/g/data/u46/fxz547/anaconda2/lib/python27.zip
/g/data/u46/fxz547/anaconda2/lib/python2.7
/g/data/u46/fxz547/anaconda2/lib/python2.7/plat-linux2
/g/data/u46/fxz547/anaconda2/lib/python2.7/lib-tk
/g/data/u46/fxz547/anaconda2/lib/python2.7/lib-old
/g/data/u46/fxz547/anaconda2/lib/python2.7/lib-dynload
/home/547/fxz547/.local/lib/python2.7/site-packages
/g/data1/u46/fxz547/anaconda2/lib/python2.7/site-packages/Sphinx-1.3.5-py2.7.egg
/g/data/u46/fxz547/anaconda2/lib/python2.7/site-packages
/g/data/u46/fxz547/anaconda2/lib/python2.7/site-packages/cryptography-1.0.2-py2.7-linux-x86_64.egg
/g/data/u46/fxz547/anaconda2/lib/python2.7/site-packages/IPython/extens

In [2]:

%matplotlib inline

from datacube.api import API, make_mask, list_flag_names

import xarray as xr

import sys
sys.path.insert(0,"/g/data1/u46/fxz547/Githubz/agdc-v2")

# cp /g/data/v10/public/modules/agdc-py2-prod/1.0.2/datacube.conf ~/.datacube.conf

dc = API()

CELL = (19, -29)

/g/data1/u46/fxz547/Githubz/agdc-v2/datacube/api/_api.py:64: UserWarning: No application name set during API initialization
  warnings.warn('No application name set during API initialization')


# Pixel level data filtering
This notebook steps through how to filter data using the low level Datacube bit-masking API functions.

## Load PQ Dataset (by cell)

Load the pixel quality data, for all available times, in a particular cell, into an `xarray.Dataset`.

In [3]:
pqa = dc.get_dataset_by_cell(CELL, product='pqa', platform='LANDSAT_8')
pqa

<xarray.Dataset>
Dimensions:       (time: 61, x: 4000, y: 4000)
Coordinates:
  * time          (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y             (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...
  * x             (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...
Data variables:
    crs           int64 0
    pixelquality  (time, y, x) int16 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
Attributes:
    license: Creative Commons Attribution 4.0 International (CC-BY 4.0)
    title: Experimental Data files From the Australian Geoscience Data Cube - DO NOT USE
    product_version: 0.0.0
    summary: These files are experimental, short lived, and the format will change.
    source: This data is a reprojection and retile of Landsat surface reflectance scene data.
    storage_type: ls8_pq_albers

## Describe the options available with the PQ

Print out a listing of the available bit-mask flags in this bit-masked dataset.

In [4]:
print(describe_flags(pqa))

NameError: name 'describe_flags' is not defined

In [5]:
list_flag_names(pqa)

[u'band_1_saturated',
 u'band_2_saturated',
 u'band_3_saturated',
 u'band_4_saturated',
 u'band_5_saturated',
 u'band_6_1_saturated',
 u'band_6_2_saturated',
 u'band_7_saturated',
 u'cloud_acca',
 u'cloud_fmask',
 u'cloud_shadow_acca',
 u'cloud_shadow_fmask',
 u'contiguity',
 u'land_obs',
 u'sea_obs']

## Define good pixel dict

After looking at the available flags for this type of dataset, we need to define which of them we care about when masking our pixel. We define this with a dictionary of flag name to boolean value.

In [6]:
GA_GOOD_PIXEL = {name: False for name in ('band_5_saturated',
 'band_6_1_saturated',
 'cloud_shadow_acca',
 'cloud_fmask',
 'band_3_saturated',
 'band_1_saturated',
 'band_4_saturated',
 'band_2_saturated',
 'cloud_acca',
 'band_6_2_saturated',
 'cloud_shadow_fmask',
 'band_7_saturated')}

GA_GOOD_PIXEL.update(dict(contiguity=True, land_obs=True))  #additinal key-value pairs to the dictionary GA_GOOD_P

print GA_GOOD_PIXEL

{'cloud_shadow_acca': False, 'cloud_acca': False, 'land_obs': True, 'band_1_saturated': False, 'contiguity': True, 'band_2_saturated': False, 'band_3_saturated': False, 'band_6_1_saturated': False, 'band_7_saturated': False, 'cloud_fmask': False, 'cloud_shadow_fmask': False, 'band_6_2_saturated': False, 'band_4_saturated': False, 'band_5_saturated': False}


## Make a mask to get only good pixels

Now we have our Dataset of pixel quality data, layered by time and we have a dictionary describing which bits we care about in that Dataset. We need to combine the two to create a mask. This is done using the `make_mask` function available in the Datacube API.

In [9]:
good_pixel_mask = make_mask(pqa, **GA_GOOD_PIXEL)
good_pixel_mask

KeyError: 'values'

## Load NBAR Dataset (by Cell)

Now we retrieve the NBAR data from Landsat 8 for bands 1, 2 and 3 for the same cell from which we retrieved the pixel quality data.

In [7]:
nbar = dc.get_dataset_by_cell(CELL, variables=('band_1', 'band_2', 'band_3'), product='nbar', platform='LANDSAT_8')
nbar

<xarray.Dataset>
Dimensions:  (time: 61, x: 4000, y: 4000)
Coordinates:
  * time     (time) datetime64[ns] 2013-03-27T23:45:53.380000 ...
  * y        (y) float64 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 -2.8e+06 ...
  * x        (x) float64 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 1.9e+06 ...
Data variables:
    crs      int64 0
    band_2   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
    band_3   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
    band_1   (time, y, x) int16 -999 -999 -999 -999 -999 -999 -999 -999 -999 ...
Attributes:
    license: Creative Commons Attribution 4.0 International (CC-BY 4.0)
    title: Experimental Data files From the Australian Geoscience Data Cube - DO NOT USE
    product_version: 0.0.0
    summary: These files are experimental, short lived, and the format will change.
    source: This data is a reprojection and retile of Landsat surface reflectance scene data.
    storage_type: ls8_nbar_albers

## Apply the mask to the surface reflection data


The loaded data can now be masked using the [`xarray.Dataset.where`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.where.html) function, which uses the co-ordinate values in both the mask and the data to ensure that they align. It will remove any data for which the mask is not available, and will throw an error if the mask and data do not overlap.

In [8]:
nbar_good_pixels = nbar.where(good_pixel_mask)   #['band_1']
nbar_good_pixels

NameError: name 'good_pixel_mask' is not defined

In [11]:

nbar_good_pixels['band_1'][60].plot.imshow()

NameError: name 'nbar_good_pixels' is not defined

In [12]:
nbar_good_pixels.band_2[5].plot()


NameError: name 'nbar_good_pixels' is not defined